# Training Pipeline using Title and Body Model

In [30]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from nltk.tokenize import word_tokenize
from toolbox.data_prep_helpers import *
from toolbox.evaluation import *

#from models.lstm_classifier import create_model
from models.title_body_lstm import create_model as tb_create_model

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
import numpy as np
import time



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_path = "../data/pythonquestions/"
ft_path = "sg_model.ft"  # set this to None if you want to train your own fasttext embeddings
n_top_labels = 100
n_epochs = 300
max_question_words = 100
sample_size = -1  # set to -1 to use entire data
normalize_embeddings = True
use_titles = False

tokenized_field = "q_all_body_tokenized"
content_field = "Body_q"

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dschr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Data

In [4]:
df = load_data(data_path, ignore_cache=False, tokenized_field=tokenized_field, content_field=content_field)
df.shape

loading data from cached pickle


(607282, 5)

In [5]:
sample = df.sample(sample_size) if sample_size > 0 else df
del df
sample.head()

,Id,Title,Body_q,tags,q_all_body_tokenized
0,469,How can I find the full path to a font from it...,I am using the Photoshop's javascript API to f...,"[python, osx, fonts, photoshop]","[i, am, using, the, photoshop, 's, javascript,..."
1,502,Get a preview JPEG of a PDF on Windows?,I have a cross-platform (Python) application w...,"[python, windows, image, pdf]","[i, have, a, cross-platform, (, python, ), app..."
2,535,Continuous Integration System for a Python Cod...,I'm starting work on a hobby project with a py...,"[python, continuous-integration, extreme-progr...","[i, 'm, starting, work, on, a, hobby, project,..."
3,594,cx_Oracle: How do I iterate over a result set?,There are several ways to iterate over a resul...,"[python, sql, database, oracle, cx-oracle]","[there, are, several, ways, to, iterate, over,..."
4,683,Using 'in' to match an attribute of Python obj...,I don't remember whether I was dreaming or not...,"[python, arrays, iteration]","[i, do, n't, remember, whether, i, was, dreami..."


In [6]:
# remove_html_tags(chunk, ["Body_q"])
#print(f"{i}: generating question level tokens")
sample["q_title_tokenized"] = sample["Title"].apply(generate_question_level_tokens)

### Clean Data

In [9]:
sample.shape

(607282, 6)

In [7]:
# we have some nans in our tags which break target encoding
print(sample.shape)
sample = sample[sample["tags"].apply(lambda tags: all([isinstance(t, str) for t in tags]))]
print(sample.shape)


# Reduce the number of tags and adjust dataframe accordingly
sample = reduce_number_of_tags(sample, n_top_labels)
sample.shape

(607282, 6)
(606841, 6)
deleting element python from top_tags


(425658, 6)

In [9]:
sample["tags"].head(5)

0               [osx]
1    [windows, image]
3     [sql, database]
4            [arrays]
5       [django, oop]
Name: tags, dtype: object

### Prepare Training and Test data

In [10]:
# Tokenize text into words on question level
data = sample[sample[tokenized_field].apply(len) <= max_question_words]
data = data[data["q_all_body_tokenized"].apply(len) > 0]

# train_data, test_data = train_test_split(data, test_size = 0.2)
# print(train_data.shape)
# print(test_data.shape)

In [13]:
data.shape

(97627, 6)

In [11]:
# train word embeddings ONLY with training data
# wv = create_Word2Vec_embeddings(train_data, "Body_q")
# Use FastText to include solution for out-of-vocab words
if ft_path is not None:
    wv = load_fasttext_embeddings(ft_path)
else:
    wv = create_FastText_embeddings(df, content_field)   
wv.init_sims()

## Train with Title and Body

In [12]:
X_t = data["q_title_tokenized"].apply(lambda x: np.array([wv.word_vec(w, use_norm=normalize_embeddings) for w in x]))
X_b = data["q_all_body_tokenized"].apply(lambda x: np.array([wv.word_vec(w, use_norm=normalize_embeddings) for w in x]))

padding_element = np.array([0.0] * X_t.iloc[0].shape[-1])

X_t = pad_sequences(X_t, padding="post", dtype='float32', value=padding_element)
X_b = pad_sequences(X_b, padding="post", dtype='float32', value=padding_element)
print(X_t.shape)
print(X_b.shape)

label_encoder = MultiLabelBinarizer()
label_encoder.fit(data["tags"])
y = label_encoder.transform(data["tags"])
print(y.shape)

(97627, 40, 100)
(97627, 100, 100)
(97627, 100)


### Grid Search

In [12]:
from toolbox.training import grid_search_es

search_params = {
    # conduct big grid search with these params
    "lstm_layer_size": [256,128],
    "lstm_dropout": [0.0,0.2,0.4],
    "num_mid_dense": [1, 0],
    
    # test grid search with these params (comment out for actual run)
    #"lstm_layer_size": [16],
    #"lstm_dropout": [0.0],
    #"num_mid_dense": [1, 0],
    # don#t change
    "output_dim": [y.shape[-1]]
}

all_hists = grid_search_es([X_b, X_t], y, tb_create_model, search_params, max_epochs=150)



{'lstm_layer_size': 256, 'lstm_dropout': 0.0, 'num_mid_dense': 1, 'output_dim': 100}
Train on 78101 samples, validate on 19526 samples
Epoch 1/150
  128/78101 [..............................] - ETA: 30:02 - loss: 0.0694 - accuracy: 0.9855 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00Train on 78101 samples, validate on 19526 samples
Epoch 1/150
  128/78101 [..............................] - ETA: 32:47 - loss: 0.0673 - accuracy: 0.9861 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00average min val_loss: 0.06622259205196354 -- epochs: [98, 90] -- time: 1027.04 seconds
{'lstm_layer_size': 256, 'lstm_dropout': 0.0, 'num_mid_dense': 0, 'output_dim': 100}
Train on 78101 samples, validate on 19526 samples
Epoch 1/150
  128/78101 [..............................] - ETA: 30:45 - loss: 0.0760 - accuracy: 0.9861 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00Train on 78101 samples, validate on 19526 samples
Epoch 1/150
  128/78101 [..............................] - ETA: 29:15 - loss: 0.0824

MemoryError: 

### Model Parameter

We set our model-paramter to the results of the grid search. Feel free to run it on your own or set parameters manually.

In [22]:
# Chooses the parameter configuration, which lead to the lowest loss during the grid search
best_params, best_hist, best_loss = min(all_hists, key=lambda x: x[2])

epoch_lengths = [len(h["val_loss"]) for h in best_hist]
print(f"best combindation: {best_params}")
print(f"avg min val_loss: {best_loss} -- epoch counts: {epoch_lengths}")

best combindation: {'lstm_layer_size': 256, 'lstm_dropout': 0.0, 'num_mid_dense': 1, 'output_dim': 100}
avg min val_loss: 0.06613914214908728 -- epoch counts: [100, 100]


### Create and Train Model

In [23]:
model = tb_create_model(**best_params)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, None, 100)    0           input_1[0][0]                    
__________________________________________________________________________________________________
masking_1 (Masking)             (None, None, 100)    0           input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
# If you are working on linux or windows, pay attention to the file paths "/" and "\\" respectively

import datetime

model_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir="logs\\fit\\" + model_name

callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, verbose=0),
    TensorBoard(log_dir=log_dir, histogram_freq=1),
    ModelCheckpoint(filepath=f"checkpoints\\{model_name}", monitor="val_loss", restore_best_weights=True, verbose=0)
]

# Train and test split with two datasets. Requires the zip to make the split for the question body and the question title
X_train_z, X_test_z, y_train, y_test = train_test_split(list(zip(X_b, X_t)), y, test_size=0.2)
X_train = list(zip(*X_train_z))
X_test = list(zip(*X_test_z))
model.fit(x=X_train, y=y_train, batch_size=128, epochs = 300, validation_data=[X_test, y_test], callbacks=callbacks)

Train on 78101 samples, validate on 19526 samples
Epoch 1/300
  128/78101 [..............................] - ETA: 2:01 - loss: 0.0687 - accuracy: 0.9859

In [ ]:

predictions = model.predict([X_test_t_padded, X_test_b_padded])

l_pred = label_encoder.inverse_transform(binarize_model_output(predictions, threshold=0.10))
l_true = label_encoder.inverse_transform(y_test)
texts = test_data[tokenized_field]
raw_texts = test_data[content_field]
titles = test_data["Title"]

for pred, act, txt, raw_txt, title in zip(l_pred, l_true, texts, raw_texts, titles):
    print(f"TRUE: {act}\nPREDICTION: {pred}\n")
    print(f"{title}\n----------")
    print(raw_txt)

In [59]:
predictions = model.predict([X_test_t_padded, X_test_b_padded], batch_size=64)
output_evaluation(model, sample_size, max_question_words, n_top_labels, y_test, predictions, normalize_embeddings, 1, None, n_epochs)

Model Evaluation

normalize_embeddings = True, learning_rate = 1, vocab_size = None, epochs=30
Parameter Settings:
 Sample size = -1, Max. number of words per question = 100, Number of Top Labels used = 100

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 100)]  0                                            
__________________________________________________________________________________________________
masking (Masking)               (None, None, 100)    0           input_1[0][0]                    
____________________________________________________________________________________

C:\Users\dschr\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\dschr\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Metrics with optimized threshold of 0.43
 Macro Evaluation: f1_Score= 0.5090096015650044 , Recall = 0.44893302869566176 , Precision = 0.6111371903996836
 Micro Evaluation: f1_Score= 0.5972185778011021 , Recall = 0.5130362912314975 , Precision = 0.7144501412577169


In [55]:
print(predictions.shape)
print(y_test.shape)

(19526, 100)
(19526, 100)
